In [1]:
from antifraud import read_adj_dict_from_file, adjacency_dict
from graph_algorithms import Graph

# Read files

In [2]:
%%time

batch  = '../paymo_input/batch_payment.txt'
stream = '../paymo_input/stream_payment.txt'

adj_list   = read_adj_dict_from_file (batch)
adj_stream = read_adj_dict_from_file (stream)

CPU times: user 17.8 s, sys: 366 ms, total: 18.1 s
Wall time: 18.1 s


In [3]:
graph = Graph (adj_list)

# Features

## Feature 1
When anyone makes a payment to another user, they'll be notified if they've never made a transaction with that user before.

"unverified: You've never had a transaction with this user before. Are you sure you would like to proceed with this payment?"

## Feature 2

The PayMo team is concerned that these warnings could be annoying because there are many users who haven't had transactions, but are still in similar social networks.

"unverified: This user is not a friend or a "friend of a friend". Are you sure you would like to proceed with this payment?"

## Feature 3

More generally, PayMo would like to extend this feature to larger social networks. Implement a feature to warn users only when they're outside the "4th degree friends network."